In [5]:
ACTIVITIES = {
    1: 'WALKING'            ,
    2: 'WALKING_UPSTAIRS'   ,
    3: 'WALKING_DOWNSTAIRS' ,
    4: 'SITTING'            ,
    5: 'STANDING'           ,
    6: 'LAYING'             ,
}

train_path = "./UCI HAR Dataset/train"
test_path = "./UCI HAR Dataset/test"

In [3]:
import os
import numpy as np
import pandas as pd

np.random.seed(2)